## Semantic Segmentation : U-Net

# Imports and Packages

In [ ]:
H = 256
W = 256

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np


# Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# imgs path "/content/drive/MyDrive/ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input"
# masks path "/content/drive/MyDrive/ISIC2018_Task1_Training_GroundTruth"

# Model Building

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model

In [ ]:
import tensorflow as tf

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [ ]:
inputs = Input((256, 256, 3))

""" Encoder """
s1, p1 = encoder_block(inputs, 64)
s2, p2 = encoder_block(p1, 128)
s3, p3 = encoder_block(p2, 256)
s4, p4 =encoder_block(p3, 400)
s5, p5 =encoder_block(p4, 800)
s6, p6=encoder_block(p5,1400)

""" Bridge """
b1 = conv_block(p6, 1024)

""" Decoder """
d1 = decoder_block(b1, s6, 1400)
d2 = decoder_block(d1, s5, 800)
d3 = decoder_block(d2, s4, 400)
d4 = decoder_block(d3, s3, 256)
d5 = decoder_block(d4, s2, 128)
d6 = decoder_block(d5, s1, 64)

""" Outputs """
outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d6)

""" Model """
model = Model(inputs, outputs)

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15

def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.metrics import Accuracy,Recall,Precision
from tensorflow.keras.optimizers import Adam


metrics = [dice_coef, iou, Recall(), Precision()]


In [ ]:
model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=metrics)

In [ ]:
model.summary()

# Data Preperation

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y


def load_data(dataset_path, split=0.2):
    images = sorted(glob(os.path.join(dataset_path, "ISIC2018_Task1-2_Training_Input/ISIC2018_Task1-2_Training_Input", "*.jpg")))
    masks = sorted(glob(os.path.join(dataset_path, "ISIC2018_Task1_Training_GroundTruth", "*.png")))

    test_size = int(len(images) * split)

    train_x, valid_x = train_test_split(images, test_size=test_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=test_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [ ]:
def read_image(path):
    path = path.decode()  #  converts the path argument from a bytes object to a string
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (H, W, 3) #. The cv2.IMREAD_COLOR flag indicates that the image should be loaded in the RGB color format, and not as grayscale or with alpha channel information
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x                                ## (256, 256, 3)

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (H, W)
    x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)                    ## (256, 256)
    x = np.expand_dims(x, axis=-1)              ## (256, 256, 1)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

def tf_dataset(X, Y, batch):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(10)dimension (256,256,3)
    return dataset

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)
    tf.random.set_seed(42)

    """ Folder for saving data """
    create_dir("/content/drive/MyDrive/UNET")

    """ Hyperparameters """
    batch_size = 20
    lr = 1e-2 ## (0.0001)
    num_epoch = 10
    model_path = "/content/drive/MyDrive/UNIT/model_checkpoint.h5"
    csv_path = "/content/drive/MyDrive/UNIT/data_checkpoint.csv"

    """ Dataset : 60/20/20 """
    dataset_path = "/content/drive/MyDrive"
    (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(dataset_path)

    print(f"Train: {len(train_x)} - {len(train_y)}")
    print(f"Valid: {len(valid_x)} - {len(valid_y)}")
    print(f"Test: {len(test_x)} - {len(test_y)}")

    train_dataset = tf_dataset(train_x, train_y, batch_size)
    valid_dataset = tf_dataset(valid_x, valid_y, batch_size)

    train_steps = len(train_x)//batch_size
    valid_steps = len(valid_x)//batch_size

Train: 1558 - 1558
Valid: 518 - 518
Test: 518 - 518


In [ ]:
    callbacks = [
        ModelCheckpoint(model_path, verbose=1, save_best_only=True),
        #ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
        CSVLogger(csv_path),
        TensorBoard(),
        EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    ]

In [ ]:
model.load_weights('/content/drive/MyDrive/files/model.h5')

In [ ]:
    #50-54
model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.0781 - recall: 0.9121 - precision: 0.9380 
Epoch 1: val_loss improved from inf to 0.23392, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5
78/78 [==============================] - 5627s 73s/step - loss: 0.0781 - recall: 0.9121 - precision: 0.9380 - val_loss: 0.2339 - val_recall: 0.8483 - val_precision: 0.8787
Epoch 2/20
78/78 [==============================] - ETA: 0s - loss: 0.0780 - recall: 0.9109 - precision: 0.9367 
Epoch 2: val_loss did not improve from 0.23392
78/78 [==============================] - 3713s 48s/step - loss: 0.0780 - recall: 0.9109 - precision: 0.9367 - val_loss: 0.3717 - val_recall: 0.9388 - val_precision: 0.7445
Epoch 3/20
78/78 [==============================] - ETA: 0s - loss: 0.0701 - recall: 0.9201 - precision: 0.9436
Epoch 3: val_loss improved from 0.23392 to 0.19173, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5
78/78 [==============================] - 10

In [ ]:
model.save('/content/drive/MyDrive/UNET/my_model_20oct.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
from tensorflow.keras.metrics import Accuracy,Recall,Precision
from tensorflow.keras.optimizers import Adam


metrics = [dice_coef, iou, Recall(), Precision()]



In [ ]:
new_model = tf.keras.models.load_model('/content/drive/MyDrive/UNIT/model_checkpoint.h5', custom_objects={'dice_coef': dice_coef, 'iou':iou})

In [ ]:
    #55-70
new_model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.0628 - recall: 0.9303 - precision: 0.9467 
Epoch 1: val_loss improved from inf to 0.22538, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


78/78 [==============================] - 2145s 24s/step - loss: 0.0628 - recall: 0.9303 - precision: 0.9467 - val_loss: 0.2254 - val_recall: 0.8820 - val_precision: 0.8545
Epoch 2/20
78/78 [==============================] - ETA: 0s - loss: 0.0792 - recall: 0.9069 - precision: 0.9401 
Epoch 2: val_loss did not improve from 0.22538
78/78 [==============================] - 1908s 24s/step - loss: 0.0792 - recall: 0.9069 - precision: 0.9401 - val_loss: 1.1661 - val_recall: 0.9581 - val_precision: 0.5547
Epoch 3/20
78/78 [==============================] - ETA: 0s - loss: 0.1027 - recall: 0.8795 - precision: 0.9216
Epoch 3: val_loss did not improve from 0.22538
78/78 [==============================] - 809s 10s/step - loss: 0.1027 - recall: 0.8795 - precision: 0.9216 - val_loss: 0.4098 - val_recall: 0.9250 - val_precision: 0.7651
Epoch 4/20
78/78 [==============================] - ETA: 0s - loss: 0.0757 - recall: 0.9175 - precision: 0.9336
Epoch 4: val_loss improved from 0.22538 to 0.22085, sa

UnknownError: ignored

In [ ]:
new_model.compile(loss="binary_crossentropy", optimizer=Adam(0.0001), metrics=metrics)

In [ ]:
#70-80
new_model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        callbacks=callbacks
    )


Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.0499 - dice_coef: 0.9263 - iou: 0.8629 - recall_3: 0.9448 - precision_3: 0.9561
Epoch 1: val_loss did not improve from 0.17883
78/78 [==============================] - 746s 8s/step - loss: 0.0499 - dice_coef: 0.9263 - iou: 0.8629 - recall_3: 0.9448 - precision_3: 0.9561 - val_loss: 0.2173 - val_dice_coef: 0.8669 - val_iou: 0.7673 - val_recall_3: 0.8615 - val_precision_3: 0.9007
Epoch 2/20
78/78 [==============================] - ETA: 0s - loss: 0.0446 - dice_coef: 0.9335 - iou: 0.8755 - recall_3: 0.9506 - precision_3: 0.9601
Epoch 2: val_loss did not improve from 0.17883
78/78 [==============================] - 690s 9s/step - loss: 0.0446 - dice_coef: 0.9335 - iou: 0.8755 - recall_3: 0.9506 - precision_3: 0.9601 - val_loss: 0.2145 - val_dice_coef: 0.8564 - val_iou: 0.7510 - val_recall_3: 0.8343 - val_precision_3: 0.9126
Epoch 3/20
78/78 [==============================] - ETA: 0s - loss: 0.0429 - dice_coef: 0.9360 - i

In [ ]:
#86-88
new_model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.0522 - dice_coef: 0.9219 - iou: 0.8554 - recall_1: 0.9413 - precision_1: 0.9544 
Epoch 1: val_loss improved from inf to 0.17627, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


78/78 [==============================] - 2548s 31s/step - loss: 0.0522 - dice_coef: 0.9219 - iou: 0.8554 - recall_1: 0.9413 - precision_1: 0.9544 - val_loss: 0.1763 - val_dice_coef: 0.8633 - val_iou: 0.7613 - val_recall_1: 0.8680 - val_precision_1: 0.9008
Epoch 2/20
78/78 [==============================] - ETA: 0s - loss: 0.0524 - dice_coef: 0.9215 - iou: 0.8547 - recall_1: 0.9416 - precision_1: 0.9527 
Epoch 2: val_loss did not improve from 0.17627
78/78 [==============================] - 3105s 40s/step - loss: 0.0524 - dice_coef: 0.9215 - iou: 0.8547 - recall_1: 0.9416 - precision_1: 0.9527 - val_loss: 0.2198 - val_dice_coef: 0.8626 - val_iou: 0.7605 - val_recall_1: 0.8801 - val_precision_1: 0.8798
Epoch 3/20
 4/78 [>.............................] - ETA: 30:09 - loss: 0.0628 - dice_coef: 0.9176 - iou: 0.8479 - recall_1: 0.9022 - precision_1: 0.9742

In [ ]:
#91-100
new_model.fit(
        train_dataset,
        epochs=20,
        validation_data=valid_dataset,
        callbacks=callbacks
    )

Epoch 1/20
78/78 [==============================] - ETA: 0s - loss: 0.0518 - dice_coef: 0.9231 - iou: 0.8575 - recall_1: 0.9432 - precision_1: 0.9531 
Epoch 1: val_loss improved from inf to 0.25114, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


78/78 [==============================] - 2657s 25s/step - loss: 0.0518 - dice_coef: 0.9231 - iou: 0.8575 - recall_1: 0.9432 - precision_1: 0.9531 - val_loss: 0.2511 - val_dice_coef: 0.8597 - val_iou: 0.7558 - val_recall_1: 0.8678 - val_precision_1: 0.8830
Epoch 2/20
78/78 [==============================] - ETA: 0s - loss: 0.0494 - dice_coef: 0.9267 - iou: 0.8636 - recall_1: 0.9446 - precision_1: 0.9561 
Epoch 2: val_loss improved from 0.25114 to 0.20973, saving model to /content/drive/MyDrive/UNIT/model_checkpoint.h5
78/78 [==============================] - 2211s 28s/step - loss: 0.0494 - dice_coef: 0.9267 - iou: 0.8636 - recall_1: 0.9446 - precision_1: 0.9561 - val_loss: 0.2097 - val_dice_coef: 0.8601 - val_iou: 0.7568 - val_recall_1: 0.8195 - val_precision_1: 0.9333
Epoch 3/20
78/78 [==============================] - ETA: 0s - loss: 0.0466 - dice_coef: 0.9302 - iou: 0.8696 - recall_1: 0.9481 - precision_1: 0.9579 
Epoch 3: val_loss did not improve from 0.20973
78/78 [================